In [1]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple

[    0.00] Initializing mpi-sppy


In [2]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 48  # Number of execution scenarios                     (chi)

M = 1e3  # Big M 

In [3]:
def build_model(cap_factor):
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=75, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    com1_loc3_out = Resource(name='com1_loc3_out', label='Commodity 1 sent out from location 3')
    com1_loc4_out = Resource(name='com1_loc4_out', label='Commodity 1 sent out from location 4')
    com1_loc5_out = Resource(name='com1_loc5_out', label='Commodity 1 sent out from location 5')
    com1_loc6_out = Resource(name='com1_loc6_out', label='Commodity 1 sent out from location 6')
    com1_loc7_out = Resource(name='com1_loc7_out', label='Commodity 1 sent out from location 7')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    prod_max = {0:30, 1: 60, 2: 90, 3: 120, 4: 150}
    prod_min = {0:0, 1: 30, 2: 60, 3: 90, 4: 120}
    rate_max = {0:1.25, 1: 1, 2: 0.75, 3: 0.5, 4: 0.25}
    mode_ramp = {(0,1): 5, (1,2): 5}
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 125
    
    com1_procure = Process(name='procure com1', prod_max=125, conversion={com1_pur: -1, com1_in: 1}, capex=0.01, vopex=0.01, prod_min=125, label='Procure com1')
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Sell com1')
    com1_opt_procure = Process(name='procure optional com1', prod_max=125, conversion={com1_pur: -1, com1_in:1}, capex=10, vopex=0.1, prod_min=0.01, label='Procure optional com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 1')
    com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 2')
    com1_receive_loc3 = Process(name='com1_receive_loc3', prod_max=com1_process_capacity, conversion={com1_loc3_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 3')
    com1_receive_loc4 = Process(name='com1_receive_loc4', prod_max=com1_process_capacity, conversion={com1_loc4_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 4')
    com1_receive_loc5 = Process(name='com1_receive_loc5', prod_max=com1_process_capacity, conversion={com1_loc5_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 5')
    com1_receive_loc6 = Process(name='com1_receive_loc6', prod_max=com1_process_capacity, conversion={com1_loc6_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 6')
    com1_receive_loc7 = Process(name='com1_receive_loc7', prod_max=com1_process_capacity, conversion={com1_loc7_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 7')
    
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Process the commodity through the location')
    
    # com1_process = Process(name='com1_process', prod_max=prod_max, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}, 3:{com1_in: -1, com1_out: 1}, 4:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=prod_min, rate_max=rate_max, varying=[VaryingProcess.DETERMINISTIC_CAPACITY], label='Process the commodity through the location')
    
    com1_store10 = Process(name='com1_store10', prod_max=com1_process_capacity, capex=100, vopex=1, store_min=0.01, store_max= 20, prod_min=com1_process_capacity, label="Storage capacity of 10 units", storage=com1_in, storage_cost=0.02)
    com1_store20 = Process(name='com1_store20', prod_max=com1_process_capacity, capex=200, vopex=2, store_min=0.01,store_max= 40, prod_min=com1_process_capacity, label="Storage capacity of 20 units", storage=com1_in, storage_cost=0.02)
    com1_store50 = Process(name='com1_store50', prod_max=com1_process_capacity, capex=500, vopex=5, store_min=0.01, store_max= 100, prod_min=com1_process_capacity, label="Storage capacity of 50 units", storage=com1_in, storage_cost=0.02)
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 1')
    com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 2')
    com1_loc3_send = Process(name='com1_loc3_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc3_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 3')
    com1_loc4_send = Process(name='com1_loc4_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc4_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 4')
    com1_loc5_send = Process(name='com1_loc5_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc5_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 5')
    com1_loc6_send = Process(name='com1_loc6_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc6_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 6')
    com1_loc7_send = Process(name='com1_loc7_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc7_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 7')
    
    
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_receive_loc2, com1_receive_loc3, com1_process, com1_store20, com1_loc1_send}, label="Location 1", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc2 = Location(name='loc2', processes={com1_receive_loc1, com1_receive_loc4, com1_receive_loc5, com1_process, com1_store20, com1_loc2_send}, label="Location 2", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc3 = Location(name='loc3', processes={com1_receive_loc1, com1_receive_loc4, com1_process, com1_store20, com1_loc3_send}, label="Location 3", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc4 = Location(name='loc4', processes={com1_receive_loc2, com1_receive_loc3, com1_receive_loc6, com1_receive_loc5, com1_receive_loc7, com1_process, com1_store50, com1_loc4_send, com1_opt_procure}, label="Location 4", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc5 = Location(name='loc5', processes={com1_receive_loc2, com1_receive_loc4, com1_receive_loc7, com1_process, com1_store50, com1_loc5_send, com1_sell}, label="Location 5", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc6 = Location(name='loc6', processes={com1_procure, com1_receive_loc4, com1_process, com1_store10, com1_loc6_send}, label="Location 6", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_procure: cap_factor[['com1_procure']]})
    
    loc7 = Location(name='loc7', processes={com1_receive_loc4, com1_receive_loc5, com1_process, com1_store20, com1_loc7_send}, label="Location 7", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    
    # ======================================================================================================================
    # Declare transport/trucks
    # ======================================================================================================================
    
    truck_cap12 = 70
    truck_cap13 = 30
    truck_cap24 = 50
    truck_cap25 = 30
    truck_cap34 = 30
    truck_cap45 = 100
    truck_cap47 = 40
    truck_cap64 = 50
    truck_cap75 = 40
    
    truck12 = Transport(name='truck12', resources={com1_loc1_out}, trans_max=truck_cap12, label='Truck from location 1 to 2', capex=0.5, vopex=0.05, trans_min=truck_cap12)
    truck21 = Transport(name='truck21', resources={com1_loc2_out}, trans_max=truck_cap12, label='Truck from location 2 to 1', capex=0.0001, vopex=0.05, trans_min=truck_cap12)
    
    truck13 = Transport(name='truck13', resources={com1_loc1_out}, trans_max=truck_cap13, label='Truck from location 1 to 3', capex=0.3, vopex=0.03, trans_min=truck_cap13)
    truck31 = Transport(name='truck31', resources={com1_loc3_out}, trans_max=truck_cap13, label='Truck from location 3 to 1', capex=0.0001, vopex=0.03, trans_min=truck_cap13)
    
    truck24 = Transport(name='truck24', resources={com1_loc2_out}, trans_max=truck_cap24, label='Truck from location 2 to 4', capex=0.5, vopex=0.05, trans_min=truck_cap24)
    truck42 = Transport(name='truck42', resources={com1_loc4_out}, trans_max=truck_cap24, label='Truck from location 4 to 2', capex=0.0001, vopex=0.05, trans_min=truck_cap24)
    
    truck25 = Transport(name='truck25', resources={com1_loc2_out}, trans_max=truck_cap25, label='Truck from location 2 to 5', capex=0.3, vopex=0.03, trans_min=truck_cap25)
    truck52 = Transport(name='truck52', resources={com1_loc5_out}, trans_max=truck_cap25, label='Truck from location 5 to 2', capex=0.0001, vopex=0.03, trans_min=truck_cap25)
    
    truck34 = Transport(name='truck34', resources={com1_loc3_out}, trans_max=truck_cap34, label='Truck from location 3 to 4', capex=0.2, vopex=0.02, trans_min=truck_cap34)
    truck43 = Transport(name='truck43', resources={com1_loc4_out}, trans_max=truck_cap34, label='Truck from location 4 to 3', capex=0.0001, vopex=0.02, trans_min=truck_cap34)
    
    truck45 = Transport(name='truck45', resources={com1_loc4_out}, trans_max=truck_cap45, label='Truck from location 4 to 5', capex=1, vopex=0.1, trans_min=truck_cap45)
    truck54 = Transport(name='truck54', resources={com1_loc5_out}, trans_max=truck_cap45, label='Truck from location 5 to 4', capex=0.0001, vopex=0.1, trans_min=truck_cap45)
    
    truck47 = Transport(name='truck47', resources={com1_loc4_out}, trans_max=truck_cap47, label='Truck from location 4 to 7', capex=0.4, vopex=0.04, trans_min=truck_cap47)
    truck74 = Transport(name='truck74', resources={com1_loc7_out}, trans_max=truck_cap47, label='Truck from location 7 to 4', capex=0.0001, vopex=0.04, trans_min=truck_cap47)
    
    truck64 = Transport(name='truck64', resources={com1_loc6_out}, trans_max=truck_cap64, label='Truck from location 6 to 4', capex=0.5, vopex=0.05, trans_min=truck_cap64)
    truck46 = Transport(name='truck46', resources={com1_loc4_out}, trans_max=truck_cap64, label='Truck from location 4 to 6', capex=0.0001, vopex=0.05, trans_min=truck_cap64)
    
    truck75 = Transport(name='truck75', resources={com1_loc7_out}, trans_max=truck_cap75, label='Truck from location 7 to 5', capex=0.4, vopex=0.04, trans_min=truck_cap75)
    truck57 = Transport(name='truck57', resources={com1_loc5_out}, trans_max=truck_cap75, label='Truck from location 5 to 7', capex=0.0001, vopex=0.04, trans_min=truck_cap75)
    
    
    
    # ======================================================================================================================
    # Declare network
    # ======================================================================================================================
    
    # transport_matrix = [
    #     [[], [truck50_12], [truck30_13], [], [], [], []],  # source: location 1
    #     [[truck50_21], [], [], [truck50_24], [truck30_25], [], []],  # source: location 2
    #     [[truck30_31], [], [], [truck20_34], [], [], []],  # source: location 3
    #     [[], [truck50_42], [truck20_43], [], [truck100_45], [truck50_46], [truck40_47]],  # source: location 4
    #     [[], [truck30_52], [], [truck100_54], [], [], [truck40_57]],  # source: location 5
    #     [[], [], [], [truck50_64], [], [], []],  # source: location 6
    #     [[], [], [], [truck40_74], [truck40_75], [], []]  # source: location 7
    # ]
    
    transport_matrix = [
        [[], [truck12], [truck13], [], [], [], []],  # source: location 1
        [[truck21], [], [], [truck24], [truck25], [], []],  # source: location 2
        [[truck31], [], [], [truck34], [], [], []],  # source: location 3
        [[], [truck42], [truck43], [], [truck45], [truck46], [truck47]],  # source: location 4
        [[], [truck52], [], [truck54], [], [], [truck57]],  # source: location 5
        [[], [], [], [truck64], [], [], []],  # source: location 6
        [[], [], [], [truck74], [truck75], [], []]  # source: location 7
    ]
    
    distance_matrix = [
        [0, 55, 196, M, M, M, M],
        [55, 0, M, 163, 112, M, 134],
        [196, M, 0, 63, M, M, M],
        [M, 163, 63, 0, 95, 117, 88],
        [M, 112, M, 95, 0, M, 134],
        [M, M, M, 117, M, 0, M],
        [M, 134, M, 88, 134, M, 0]
    ]
    
    locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7]
    
    sources = locset
    sinks = locset
    
    network = Network(name='Network', scales= scales, source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix, distance_matrix=distance_matrix)



    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 100
    demand_penalty = 50
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc5 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc5 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario_IS', scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, network=network, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Scenario with perfect information')
    
    # ======================================================================================================================
    # Declare problem
    # ======================================================================================================================
    
    # problem_mincost = formulate(scenario=scenario,
    #                         constraints={Constraints.COST, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
    #                         demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.TRANSPORT, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    
    scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
    capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
    
    problem_mincost.first_stage_cost  = capex_process
    
    return scenario, problem_mincost

In [4]:
#Expected Cost under Perfect Information
all_scenario_names = ["good", "average", "bad"]
ns_dict = {'good': 48, "average": 24, "bad":12}
scenario_probabilities = {'good': 0.25, 'average': 0.5, 'bad':0.25} 

exCost_PI = 0

for scenario_name in all_scenario_names:
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen, model = build_model(cap_factor=cap_factor)
    for loc in model.locations:
        model.X_P[loc,"procure com1",scen.network_scale_level].fixed= True
        model.X_P[loc,"procure com1",scen.network_scale_level].value= 1
    solver = SolverFactory("gurobi")
    solver.solve(model)
    
    exCost_PI += value(model.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint demand penalty
objective cost w demand penalty
constraint process capex
constraint process fopex
constraint process vopex
constraint process in

In [5]:
exCost_PI

295087.85

In [6]:
def scenario_creator(scenario_name):
    if scenario_name=='good':
        ns = 48
    elif scenario_name=='average':
        ns = 24
    elif scenario_name=='bad':
        ns = 12
    else:
        raise ValueError("Unrecognized scenario name")
    
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns + [0]*(_exec_scenarios-ns)})
    scen, model = build_model(cap_factor=cap_factor)
    sputils.attach_root_node(model, model.first_stage_cost, [model.X_P])
    model._mpisppy_probability = scenario_probabilities[scenario_name]
    return model

In [7]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "gurobi"}
all_scenario_names = ["good", "average", "bad"]
ef_UI = ExtensiveForm(options, all_scenario_names, scenario_creator)
results = ef_UI.solve_extensive_form()

[   96.56] Initializing SPBase
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint demand penalty
objective cost w demand penalty
constraint process capex
constraint process fopex
constraint proc

In [8]:
exCost_UI = ef_UI.get_objective_value()
print(f"{exCost_UI:.3f}")

295085.350


In [9]:
EVPI = exCost_UI - exCost_PI
EVPI

-2.5

In [10]:
soln = ef_UI.get_root_solution()
for (var_name, var_val) in soln.items():
    print(var_name, var_val)

X_P[loc1,com1_loc1_send,0] 1.0
X_P[loc1,com1_loc2_send,0] 0.0
X_P[loc1,com1_loc3_send,0] 0.0
X_P[loc1,com1_loc4_send,0] 0.0
X_P[loc1,com1_loc5_send,0] 0.0
X_P[loc1,com1_loc6_send,0] 0.0
X_P[loc1,com1_loc7_send,0] 0.0
X_P[loc1,com1_process,0] 1.0
X_P[loc1,com1_receive_loc1,0] 0.0
X_P[loc1,com1_receive_loc2,0] -0.0
X_P[loc1,com1_receive_loc3,0] -0.0
X_P[loc1,com1_receive_loc4,0] 0.0
X_P[loc1,com1_receive_loc5,0] 0.0
X_P[loc1,com1_receive_loc6,0] 0.0
X_P[loc1,com1_receive_loc7,0] 0.0
X_P[loc1,com1_store10,0] 0.0
X_P[loc1,com1_store10_discharge,0] 0.0
X_P[loc1,com1_store20,0] -0.0
X_P[loc1,com1_store20_discharge,0] 1.0
X_P[loc1,com1_store50,0] 0.0
X_P[loc1,com1_store50_discharge,0] 0.0
X_P[loc1,procure com1,0] 0.0
X_P[loc1,procure optional com1,0] 0.0
X_P[loc1,sell com1,0] 0.0
X_P[loc2,com1_loc1_send,0] 0.0
X_P[loc2,com1_loc2_send,0] 1.0
X_P[loc2,com1_loc3_send,0] 0.0
X_P[loc2,com1_loc4_send,0] 0.0
X_P[loc2,com1_loc5_send,0] 0.0
X_P[loc2,com1_loc6_send,0] 0.0
X_P[loc2,com1_loc7_send,0] 0.0

In [11]:
# from mpisppy.opt.lshaped import LShapedMethod
# 
# all_scenario_names = ["good", "average", "bad"]
# bounds = {name: -432000 for name in all_scenario_names}
# options = {
#     "root_solver": "gurobi",
#     "sp_solver": "gurobi",
#     "sp_solver_options" : {"threads" : 1},
#     "valid_eta_lb": bounds,
#     "max_iter": 10,
# }
# 
# ls = LShapedMethod(options, all_scenario_names, scenario_creator)
# result = ls.lshaped_algorithm()
# 
# variables = ls.gather_var_values_to_rank0()
# for ((scen_name, var_name), var_value) in variables.items():
#     print(scen_name, var_name, var_value)

In [12]:
n = 24
cap_factor = pandas.DataFrame(data={'com1_procure': [1]*n + [0]*(_exec_scenarios-n)})
scen, model = build_model(cap_factor=cap_factor)

for loc in model.locations:
        model.X_P[loc,"procure com1",scen.network_scale_level].fixed= True
        model.X_P[loc,"procure com1",scen.network_scale_level].value= 1

solver = SolverFactory("gurobi")
solver.solve(model)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint demand penalty
objective cost w demand penalty


{'Problem': [{'Name': 'x1', 'Lower bound': 295087.85, 'Upper bound': 295087.85, 'Number of objectives': 1, 'Number of constraints': 835058, 'Number of variables': 552653, 'Number of binary variables': 73, 'Number of integer variables': 73, 'Number of continuous variables': 552580, 'Number of nonzeros': 1127371, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.86627197265625', 'Error rc': 0, 'Time': 3.5028278827667236}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [13]:
print(f"{value(model.objective_cost_w_demand_penalty):.3f}")

295087.850


In [14]:
model.X_P.pprint()

X_P : Process Binary
    Size=168, Index=X_P_index
    Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
            ('loc1', 'com1_loc1_send', 0) :     0 :   1.0 :     1 : False : False : Binary
            ('loc1', 'com1_loc2_send', 0) :     0 :  None :     1 : False :  True : Binary
            ('loc1', 'com1_loc3_send', 0) :     0 :  None :     1 : False :  True : Binary
            ('loc1', 'com1_loc4_send', 0) :     0 :  None :     1 : False :  True : Binary
            ('loc1', 'com1_loc5_send', 0) :     0 :  None :     1 : False :  True : Binary
            ('loc1', 'com1_loc6_send', 0) :     0 :  None :     1 : False :  True : Binary
            ('loc1', 'com1_loc7_send', 0) :     0 :  None :     1 : False :  True : Binary
              ('loc1', 'com1_process', 0) :     0 :   1.0 :     1 : False : False : Binary
         ('loc1', 'com1_receive_loc1', 0) :     0 :  None :     1 : False :  True : Binary
         ('loc1', 'com1_receive_loc2', 

In [15]:
#Value of Stochastic Solution
# all_scenario_names = ["good", "average", "bad"]
# ns_dict = {'good': 12, "average": 9, "bad": 6}
exCost_FD = 0

for scenario_name in all_scenario_names:
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen, model = build_model(cap_factor=cap_factor)
    for process in scen.location_process_dict['loc1']:
        model.X_P['loc1', process, scen.network_scale_level].fixed= True

    model.X_P['loc1','com1_loc1_send',0].value = 0.0
    model.X_P['loc1','com1_process',0].value = 1.0
    model.X_P['loc1','com1_store20',0].value = 0.0
    model.X_P['loc1','com1_store20_discharge',0].value = 1.0
    model.X_P['loc1','procure com1',0].value = 1.0
    model.X_P['loc1','procure optional com1',0].value = 1.0
    model.X_P['loc1','sell com1',0].value = 1.0
    
    solver = SolverFactory("gurobi")
    solver.solve(model)
    
    exCost_FD += value(model.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint demand penalty
objective cost w demand penalty
ERROR: evaluating object as numeric value: X_P[loc1,com1_receive_loc2,0]
        (object: <class

ValueError: No value for uninitialized NumericValue object X_P[loc1,com1_receive_loc2,0]

In [ ]:
VSS = exCost_FD - exCost_UI
VSS